<a href="https://colab.research.google.com/github/zahraDehghanian97/Poetry_Generator/blob/master/Word_Poem_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np


In [2]:
seqLength = 20
BATCH_SIZE = 64
BUFFER_SIZE = 100
embedding_dim = 256
rnn_units = 1024

# make data ready

In [3]:
filepath = "/content/drive/MyDrive/Colab Notebooks/shahname.txt"
text = open(filepath, 'rb').read().decode(encoding='utf-8')
text = text.replace("\t"," \t ").replace("\n", " \n ")


In [4]:
corpusList = [w for w in text.split(' ')] 
corpus_words = [i for i in corpusList if i]
map(str.strip, corpus_words)
vocab = sorted(set(corpus_words))
print(len(corpus_words))
vocab_size = len(vocab)
word2idx = {u: i for i, u in enumerate(vocab)}
idx2words = np.array(vocab)
word_as_int = np.array([word2idx[c] for c in corpus_words])

661330


In [5]:
def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text = chunk[1:]
  return input_text, target_text

# examples_per_epoch = len(corpus_words)//(seqLength + 1)
wordDataset = tf.data.Dataset.from_tensor_slices(word_as_int)
sequencesOfWords = wordDataset.batch(seqLength + 1, drop_remainder=True)
dataset = sequencesOfWords.map(split_input_target)
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

# LSTM Model

In [6]:
def create_model_lstm(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential()
  model.add(tf.keras.layers.Embedding(vocab_size, embedding_dim,batch_input_shape=[batch_size, None]))
  model.add(tf.keras.layers.LSTM(rnn_units,return_sequences=True,stateful=True,recurrent_initializer='glorot_uniform'))
  model.add(tf.keras.layers.LSTM(rnn_units,return_sequences=True,stateful=True,recurrent_initializer='glorot_uniform'))
  model.add(tf.keras.layers.Dense(vocab_size))
  return model

In [7]:
lstm_model = create_model_lstm(vocab_size = len(vocab), embedding_dim=embedding_dim, rnn_units=rnn_units, batch_size=BATCH_SIZE)
lstm_model.compile(optimizer='adam', loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
history = lstm_model.fit(dataset, epochs=50)
main_lstm_model = create_model_lstm(vocab_size = len(vocab), embedding_dim=embedding_dim, rnn_units=rnn_units, batch_size=1)
main_lstm_model.set_weights(lstm_model.get_weights())
# main_lstm_model = tf.keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/word_lstm.h5')
main_lstm_model.summary()

Epoch 1/50
492/492 [==============================] - 71s 127ms/step - loss: 7.0382 - accuracy: 0.0740
Epoch 2/50
492/492 [==============================] - 64s 129ms/step - loss: 6.9005 - accuracy: 0.0744
Epoch 3/50
492/492 [==============================] - 64s 130ms/step - loss: 6.7166 - accuracy: 0.0746
Epoch 4/50
492/492 [==============================] - 66s 133ms/step - loss: 5.9461 - accuracy: 0.1216
Epoch 5/50
492/492 [==============================] - 66s 134ms/step - loss: 5.2907 - accuracy: 0.1891
Epoch 6/50
492/492 [==============================] - 67s 137ms/step - loss: 4.9914 - accuracy: 0.2218
Epoch 7/50
492/492 [==============================] - 67s 137ms/step - loss: 4.8003 - accuracy: 0.2335
Epoch 8/50
492/492 [==============================] - 67s 136ms/step - loss: 4.6382 - accuracy: 0.2435
Epoch 9/50
492/492 [==============================] - 67s 136ms/step - loss: 4.4908 - accuracy: 0.2521
Epoch 10/50
492/492 [==============================] - 67s 136ms/step - l

In [8]:
def generate_text(model, start_string, temperature = 0.6):
  num_generate = 200
  start_string_list =  [w for w in start_string.split(' ')]
  input_eval = [word2idx[s] for s in start_string_list]
  input_eval = tf.expand_dims(input_eval, 0)
  text_generated = []
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      predictions = tf.squeeze(predictions, 0)
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
      input_eval = tf.expand_dims([predicted_id], 0)
      text_generated.append(idx2words[predicted_id])
  return (start_string + ' '.join(text_generated))

In [9]:
print(generate_text(main_lstm_model, start_string=u"چنین گفت رستم به اسفندیار"))
main_lstm_model.save("/content/drive/MyDrive/Colab Notebooks/word_lstm.h5")

چنین گفت رستم به اسفندیار	 که ای سرفرازان و گردان نو 
 ازین پس چو برگشت و هم کینه‌جوی 	 بفرمود تا شد بپشت سپاه 
 بیامد بر لشکر نامدار 	 درفش سپهدار توران نگون 
 چو پیران چنان دید برگشت زری 	 سوی لشکر خویش بنهاد روی 
 نگه کرد پیران که خسرو بدرد 	 ز هر سو بلشکر گه آورد زود 
 چهارم کجا نیست امروز کار 	 پشیمان شدم گر بتوران شوند 
 ز ما هرک او جامهٔ هندوان 	 یکی تیز و آن نامداران کین 
 بخوردند سوگندهای گران 	 که ای پهلوان رد لهراسپ شو 
 که میخواستم کایزد دادگر 	 ندادی تو برکندی از انجمن 
 که دیوی بود 	 ز خون ریختن بازکش خویشتن 
 که من پیش خسرو بگوی 	 که ای نامور گرد ویسه نژاد 
 وز آنجا بهامون نهادند روی 	 که گفتی ز گردان توران گروه 
 طلایه سوی گاه رفتند باز 	 پذیره فرستاد و برگشت کار 
 بپیران چنین گفت پس پهلوان 	 که توران سرافراز و اسپی مکان 
 سوی پهلوان روی برگاشتند 	 غریویدن و اسب و اسب و ستام 	 ز کینه بخشم اندر آورد پای 
 همه کار بر دست او بر نهاد 	 همی بینم از
